# Create Dataframe for wind verification

Import librairies

In [3]:
import numpy as np                      # Data
import pandas as pd                     # Data 
import geopandas as gpd                 # Data
import xarray as xr                     # Data
import atlite                           # Model
import matplotlib.pyplot as plt         # Plot
from matplotlib.lines import Line2D     # Plot
from tqdm import tqdm                   # Visualise progression in loop
import yaml                             # Open yaml files

## EirGrid

### Locations of farms

Read csv file located in /Data_Final folder

In [4]:
df_capacity = pd.read_csv('../Data_Final/EirGrid/capacity_wind_9224_eir.csv',
                               index_col = 2,
                               parse_dates=True)

Rename the columns to be correctly read by Atlite

In [5]:
df_capacity = df_capacity.dropna().rename(columns={'Capacity (MW)':'capacity', 'Latitude':'y', 'Longitude':'x'})
df_capacity = df_capacity[df_capacity.index < '2024'] # Make sure to remove all data before 2024

## Atlite

### Create Cutout for Ireland

Load weather files downloaded on Copernicus website

In [6]:
ds_uv100_to2018 = xr.open_dataset('../Data/ERA5/ERA5_100m_u_v_hourly_2009_2018.nc')
ds_uv100_2019_2023 = xr.open_dataset('../Data/ERA5/ERA5_100m_u_v_hourly_2019_2023.nc')
ds_roughness_2018_2023 = xr.open_dataset('../Data/ERA5/ERA5_forecast_surface_roughness_geopotential_hourly_2018_2023.nc')
ds_roughness_to2017 = xr.open_dataset('../Data/ERA5/ERA5_forecast_surface_roughness_geopotential_hourly_2012_2017.nc')
ds_uv100_to2018 = ds_uv100_to2018.sel(time=slice("2014-01-01", "2018-12-31"))
ds_roughness_to2018 = ds_roughness_to2017.sel(time=slice("2014-01-01", "2017-12-31"))

/home/boris/anaconda3/lib/python3.11/site-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.31.0 or higher is recommended. You are running version 2.24.2
  warnings.warn(


Some variables need to be formated

In [7]:
ds_uv100_2019_2023 = ds_uv100_2019_2023.reduce(np.nansum, dim='expver',keep_attrs=True)
ds_roughness_2018_2023 = ds_roughness_2018_2023.reduce(np.nansum, dim='expver',keep_attrs=True)

In [8]:
ds_uv100_2019_2023 = ds_uv100_2019_2023.reduce(np.nansum, dim='expver',keep_attrs=True)
ds_roughness_2018_2023 = ds_roughness_2018_2023.reduce(np.nansum, dim='expver',keep_attrs=True)

ValueError: Dataset does not contain the dimensions: ['expver']

In [6]:
ds_uv100 = xr.concat([ds_uv100_to2018.rename({'lat':'latitude', 'lon':'longitude'}), ds_uv100_2019_2023], dim='time')
ds_roughness = xr.concat([ds_roughness_to2017, ds_roughness_2018_2023], dim='time')

In [7]:
ds = xr.merge([ds_uv100, ds_roughness])
ds = ds.sel(time=slice("2014-01-01", "2023-12-31"))

Loads the climate variables in Atlite

This can only be done via the function get_cutout_from_era5_data which is not originally in the Atlite scripts but has been added by the authors

In [8]:
cutout_ie = atlite.cutout.get_cutout_from_era5_data('path', ds, ['wind'])

Define function to create a time dependent layout of the capacity from the dataframe

In [9]:
def get_time_dependent_capacity_distribution(cutout_ie: atlite.Cutout, df_capacity: pd.DataFrame):
    capacity_layout = cutout_ie.data['roughness'].copy()
    capacity_layout.name = 'Capacity'
    capacity_layout[:,:,:] = 0.

    # Iterate over all capacity installations
    for idx, row in tqdm(df_capacity.reset_index().iterrows(), total= df_capacity.shape[0]):
        cap = row['capacity'] # Capacity values
        df_capacity_i = pd.DataFrame([row])
        layout = cutout_ie.layout_from_capacity_list(df_capacity_i, col="capacity")

        capacity_layout[capacity_layout['time'] >= row['Connection date']] += layout

    return capacity_layout

### Get atlite generation

In [10]:
def get_cf_series_atlite(cutout_ie: atlite.Cutout, capacity_layout, power_curve):
    if isinstance(power_curve, dict):
        # Dict contains wind_speed, power, power_max, hub_height
        wind_cf = atlite.convert.convert_wind(cutout_ie.data, power_curve)
    elif isinstance(power_curve, str):
        # Str name of turbine which atlite knows
        wind_cf = atlite.convert.convert_wind(cutout_ie.data, atlite.resource.get_windturbineconfig(power_curve))

    if isinstance(capacity_layout, xr.DataArray):
        # What supposed to happen
        return wind_cf.weighted(capacity_layout).mean(('x','y'))
    elif isinstance(capacity_layout, pd.DataFrame):
        # Take this path if layout is not processed by us
        return wind_cf.weighted(get_time_dependent_capacity_distribution(cutout_ie, capacity_layout).mean(('x','y')))

### Turbines

On an analysis of multiple turbines and their smoothing, we identify the Enercon.E112.4500 with 0.30w from renewables.ninja as the turbine with the best representation of EirGrid data

Read the Enercon Enercon.E112.4500 power curve and the smoothed version as well

In [11]:
df_turbines_ninja_nosmoothing = pd.read_csv('../Data/renewables_ninja/Wind Turbine Power Curves ~ 5 (0.01ms with 0.00 w smoother).csv')
turbine_ninja_nosmoothing = {'hub_height':100, 'P':1., 'V':df_turbines_ninja_nosmoothing['speed'].values, 'POW':df_turbines_ninja_nosmoothing['Enercon.E112.4500'].values}

df_turbines_ninja_030smoothing = pd.read_csv('../Data/renewables_ninja/Wind Turbine Power Curves ~ 5 (0.01ms with 0.30 w smoother).csv')
turbine_ninja_030smoothing = {'hub_height':100, 'P':1., 'V':df_turbines_ninja_030smoothing['data$speed'].values, 'POW':df_turbines_ninja_030smoothing['Enercon.E112.4500'].values}

### Capacity Factor

Now we calculate the wind capacity factor series for both the smoothed and non-smoothed curves. We will compare the two to EirGrid before comparing the different methods to visualise the effective difference this makes.

In [12]:
full_time_layout = get_time_dependent_capacity_distribution(cutout_ie=cutout_ie, df_capacity=df_capacity)

100%|██████████| 381/381 [01:00<00:00,  6.34it/s]


In [13]:
cf_atlite = get_cf_series_atlite(cutout_ie=cutout_ie, capacity_layout=full_time_layout, power_curve=turbine_ninja_030smoothing)
cf_atlite_nosmooth = get_cf_series_atlite(cutout_ie=cutout_ie, capacity_layout=full_time_layout, power_curve=turbine_ninja_nosmoothing)

## C3S-E Gridded

Read CF data from C3S-Energy from 2014 to 2023

In [14]:
da_gridded = xr.open_dataarray('../Data/C3S-E/c3se_onshorewind_capacityfactor_20140101_20231231_gridded_ireland.nc')

In [15]:
def get_cf_series_c3se_gridded(da_gridded: xr.DataArray, df_capacity: pd.DataFrame, only_cf: bool = True):
    lats = da_gridded['latitude'].values
    lons = da_gridded['longitude'].values

    error_farms_idx = []

    summed_time_series = np.zeros(da_gridded['time'].shape)
    total_capacity_time_series  = np.zeros(da_gridded['time'].shape)

    # Iterate over all capacity installations
    for idx, row in df_capacity.reset_index().iterrows():
        x = row['x']
        y = row['y']
        cap = row['capacity']

    # Find the nearest lat/lon point
        dif_min_lon = np.argmin(abs(lons-x))
        dif_min_lat = np.argmin(abs(lats-y))

        time_series = da_gridded[:, dif_min_lat, dif_min_lon]

        pairs_list = [[0,0], [0,1], [1,0], [1,1], [2,0], [0,2], [2,1], [1,2]]
        if np.isnan(time_series).sum() > 2:
            for pair in pairs_list:
                dif_min_lon = np.argsort(abs(lons-x))[pair[0]]
                dif_min_lat = np.argsort(abs(lats-y))[pair[1]]
                time_series = da_gridded[:, dif_min_lat, dif_min_lon]
                if np.isnan(time_series).sum() <= 2:
                    break

        capacity_time_series = time_series.copy()
        capacity_time_series[:] = cap

        time_series[time_series['time']<row['Connection date']] = 0.
        capacity_time_series[capacity_time_series['time']<row['Connection date']] = 0.

    # Add the mean CF time series to the total multiplied by the capacity (weight)
        if np.isnan(time_series).sum() <= 2:
            summed_time_series += cap*time_series
            total_capacity_time_series += capacity_time_series
        else:
            print(idx, row)

    # Divide the total time series by the total IC to go back to CF
    cf_time_series = summed_time_series/total_capacity_time_series
    if only_cf:
        return cf_time_series
    return cf_time_series, summed_time_series, total_capacity_time_series

In [16]:
cf_c3se_gridded = get_cf_series_c3se_gridded(da_gridded=da_gridded, df_capacity=df_capacity)

For C3S-Energy the wind turbine power curve is the following

In [17]:
from yaml import safe_load
path_turbines_vestas = '../Data/power_curve_Vestas_V136_3450_c3se.yaml'
with open(path_turbines_vestas, 'r') as f:
    df_turbines_vestas = pd.json_normalize(safe_load(f))
    turbine_vestas = {'hub_height':100, 'P':1, 'V':np.array(df_turbines_vestas['V'].values[0]), 'POW':np.array(df_turbines_vestas['NPOW'].values[0])}

In [18]:
cf_atlite_vestas = get_cf_series_atlite(cutout_ie=cutout_ie, capacity_layout=full_time_layout, power_curve=turbine_vestas)

## C3S-E National

Read CSV file containing Capacity Factor from C3S-E National

In [19]:
df_nat = pd.read_csv('../Data/C3S-E/c3se_onshorewind_capacityfactor_national.csv',
                         skiprows = 52,
                         usecols = [0,18], # Retrieve IE data
                         index_col = 0,
                         parse_dates = True
)

Select Time of verification (2014 - 2023)

In [20]:
df_nat = df_nat[("2014" <= df_nat.index) & (df_nat.index < "2024")]

Read CSV file containing Capacity Factor from C3S-E Subnational

In [21]:
df_sub = pd.read_csv('../Data/C3S-E/c3se_onshorewind_capacityfactor_subnational.csv',
                     skiprows = 52,
                     usecols = [0,350], # Data for NI
                     index_col = 0,
                     parse_dates = True)


Select Time of verification (2014 - 2023)

In [22]:
df_sub = df_sub[("2014" <= df_sub.index) & (df_sub.index < "2024")]

In [23]:
capacity_series_ie = get_time_dependent_capacity_distribution(cutout_ie=cutout_ie, df_capacity=df_capacity[df_capacity['ROI/NI']=='ROI'])
capacity_series_ni = get_time_dependent_capacity_distribution(cutout_ie=cutout_ie, df_capacity=df_capacity[df_capacity['ROI/NI']=='NI'])

100%|██████████| 66/66 [00:09<00:00,  6.60it/s]


Aggregate the results into one time series for each distribution

In [24]:
capacity_series_ie = capacity_series_ie.sum(dim=['x','y'])
capacity_series_ni = capacity_series_ni.sum(dim=['x','y'])

### Missing values

The C3S-E National data is missing data for the 31st of December of 2019. We add it here and make it NaN in order to facilitate further treatment.

In [25]:
df_nat.loc[pd.to_datetime('2019-12-31T22:00'), 'IE'] = np.nan
df_nat.loc[pd.to_datetime('2019-12-31T23:00'), 'IE'] = np.nan
df_sub.loc[pd.to_datetime('2019-12-31T22:00'), 'UKN0'] = np.nan
df_sub.loc[pd.to_datetime('2019-12-31T23:00'), 'UKN0'] = np.nan

Resort the data to avoid future issues

In [26]:
df_nat = df_nat.sort_index()
df_sub = df_sub.sort_index()

Finally we average the two times series with capacities as weigths

In [27]:
cf_c3se_national = (df_nat['IE']*capacity_series_ie + df_sub['UKN0']*capacity_series_ni) / (capacity_series_ie + capacity_series_ni)

## EirGrid

### Availability

In [28]:
eirgrid_qtr = pd.read_csv('../Data_Final/EirGrid/eirgrid_qtr_15min_wind_generation_availability_20140101_20231231.csv',
                          index_col = 0,
                          parse_dates = True)
eirgrid_qtr = eirgrid_qtr[("2014" <= eirgrid_qtr.index) & (eirgrid_qtr.index < "2024")]

In [29]:
cf_eirgrid = eirgrid_qtr['Availability'].resample('1h').mean() / full_time_layout.sum(dim=['x','y'])

# Save data as csv file to be used again

In [ ]:
df = pd.DataFrame({'time':cf_eirgrid.index,
                    'EirGrid':cf_eirgrid.values,
                    'Atlite': cf_atlite.values,
                    'C3S Gridded': cf_c3se_gridded.values,
                    'C3S National': cf_c3se_national.values}
                )
df = df.set_index('time')
df.to_csv('../Data/verification_cf_wind_1423.csv')